# Capstone 3: Exploratoy Data Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from time import strptime

In [2]:
exports = pd.read_excel('../data/Cleaned_Exports by Year and Country.xlsx')
yearly_shipments = pd.read_excel('../data/Cleaned_Defense Aircraft and Parts Shipments(millions of dollars).xlsx')

In [3]:
exports.head()

,Year,Value,Country,HS Code
0,2010,2575,Comoros,8411917050--PARTS OF TURBOJET & TURBOPROPELLR ...
1,2010,2629,Belize,8411917050--PARTS OF TURBOJET & TURBOPROPELLR ...
2,2010,3286,Belarus,"8411997050--PARTS OF GAS TURBINE A/C ENG, EXC ..."
3,2010,3348,Jamaica,"8411997050--PARTS OF GAS TURBINE A/C ENG, EXC ..."
4,2010,3500,Guatemala,"8411997050--PARTS OF GAS TURBINE A/C ENG, EXC ..."


In [4]:
yearly_shipments.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual,Measure
0,1992,2719,3496,3018,3433,2916,3515,2894,3125,2968,3213,3275,4453,39025,Defense Aircraft and Parts (millions of dollars)
1,1993,2456,3045,3001,2890,2874,3024,2880,2767,3257,3235,2820,3218,35467,Defense Aircraft and Parts (millions of dollars)
2,1994,2471,2698,3046,2614,2506,2829,2504,3007,2828,2802,2837,3322,33464,Defense Aircraft and Parts (millions of dollars)
3,1995,2090,2484,2604,2389,2384,2682,2366,2425,2583,2480,2433,3064,29984,Defense Aircraft and Parts (millions of dollars)
4,1996,1881,2268,2790,2164,2313,2687,2003,2713,2921,2394,2564,2995,29693,Defense Aircraft and Parts (millions of dollars)


### Prepping Exports

The HS code of this dataframe needs to be split up, with the number identifier in one column and the description in another.

In [5]:
HS_code = exports['HS Code'].str.split('--', expand = True)
HS_code.head()

,0,1
0,8411917050,"PARTS OF TURBOJET & TURBOPROPELLR A/C ENG,EX C..."
1,8411917050,"PARTS OF TURBOJET & TURBOPROPELLR A/C ENG,EX C..."
2,8411997050,"PARTS OF GAS TURBINE A/C ENG, EXC CIVIL A/C"
3,8411997050,"PARTS OF GAS TURBINE A/C ENG, EXC CIVIL A/C"
4,8411997050,"PARTS OF GAS TURBINE A/C ENG, EXC CIVIL A/C"


In [6]:
exports['HS Code Number'] = HS_code[0]
exports['HS Code Description'] = HS_code[1]
exports = exports.drop(['HS Code'], axis = 1)
exports.head()

,Year,Value,Country,HS Code Number,HS Code Description
0,2010,2575,Comoros,8411917050,"PARTS OF TURBOJET & TURBOPROPELLR A/C ENG,EX C..."
1,2010,2629,Belize,8411917050,"PARTS OF TURBOJET & TURBOPROPELLR A/C ENG,EX C..."
2,2010,3286,Belarus,8411997050,"PARTS OF GAS TURBINE A/C ENG, EXC CIVIL A/C"
3,2010,3348,Jamaica,8411997050,"PARTS OF GAS TURBINE A/C ENG, EXC CIVIL A/C"
4,2010,3500,Guatemala,8411997050,"PARTS OF GAS TURBINE A/C ENG, EXC CIVIL A/C"


In [7]:
exports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3281 entries, 0 to 3280
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Year                 3281 non-null   int64 
 1   Value                3281 non-null   int64 
 2   Country              3281 non-null   object
 3   HS Code Number       3281 non-null   object
 4   HS Code Description  3281 non-null   object
dtypes: int64(2), object(3)
memory usage: 128.3+ KB


In [8]:
#need to change the data type
exports = exports.astype({'HS Code Number':'int64', 'HS Code Description': 'string'})
exports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3281 entries, 0 to 3280
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Year                 3281 non-null   int64 
 1   Value                3281 non-null   int64 
 2   Country              3281 non-null   object
 3   HS Code Number       3281 non-null   int64 
 4   HS Code Description  3281 non-null   string
dtypes: int64(3), object(1), string(1)
memory usage: 128.3+ KB


### Prepping yearly shipments

The current layout of this dataframe is not condusive to plotting exports over the entire time period. Going to change the dataframe structure.

In [9]:
#First want to pull out the annual totals to be able to plot that separately
shipments_annualvalue = yearly_shipments[['Year', 'Annual']]
shipments_annualvalue.head()

,Year,Annual
0,1992,39025
1,1993,35467
2,1994,33464
3,1995,29984
4,1996,29693


In [10]:
#delete the two columns that we don't need anymore
yearly_shipments = yearly_shipments.drop(['Annual', 'Measure'], axis = 1)

In [11]:
yearly_shipments = yearly_shipments.melt(id_vars = 'Year', var_name = 'Month', value_name = 'Export Value (millions of dollars)')
yearly_shipments.head()

,Year,Month,Export Value (millions of dollars)
0,1992,Jan,2719
1,1993,Jan,2456
2,1994,Jan,2471
3,1995,Jan,2090
4,1996,Jan,1881


In [12]:
yearly_shipments['Month_num'] = [strptime(str(x), '%b').tm_mon for x in yearly_shipments['Month']]
yearly_shipments = yearly_shipments.drop(['Month'], axis = 1)
yearly_shipments = yearly_shipments.sort_values(['Year', 'Month_num'])
yearly_shipments.head()

,Year,Export Value (millions of dollars),Month_num
0,1992,2719,1
30,1992,3496,2
60,1992,3018,3
90,1992,3433,4
120,1992,2916,5


In [13]:
#Want to combine year and month columns, first verifying that the data each column is an integer
yearly_shipments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360 entries, 0 to 359
Data columns (total 3 columns):
 #   Column                              Non-Null Count  Dtype
---  ------                              --------------  -----
 0   Year                                360 non-null    int64
 1   Export Value (millions of dollars)  360 non-null    int64
 2   Month_num                           360 non-null    int64
dtypes: int64(3)
memory usage: 11.2 KB


In [14]:
#both Year and Month columns are integers, so combining into one column and deleting the original ones
yearly_shipments['Year'] = pd.to_datetime(yearly_shipments.Year.astype(str) + '/' + yearly_shipments.Month_num.astype(str) + '/01')
yearly_shipments.rename(columns = {'Year': 'Year-Month'}, inplace = True)
yearly_shipments = yearly_shipments.drop(['Month_num'], axis = 1)
yearly_shipments.head()

,Year-Month,Export Value (millions of dollars)
0,1992-01-01,2719
30,1992-02-01,3496
60,1992-03-01,3018
90,1992-04-01,3433
120,1992-05-01,2916


### Saving files

In [15]:
exports.to_excel('../data/EDA_exports.xlsx', index = False)
yearly_shipments.to_excel('../data/EDA_yearly_shipments.xlsx', index = False)

### Additional EDA Steps

Graphs were made for this project's EDA step in Tableau. You can view them at the link [here](https://public.tableau.com/app/profile/ashley.grindal/viz/Capstone3EDA/Sheet2?publish=yes).